# Intro
This file contains a collection of potentially useful methods, mostly from discarded approaches.

In [ ]:
import pandas as pd
def dirtify_prediction(prediction_request):
    """Adds poorly prepared data to the requested prediction, to make sure your model fail if it can't handle poorly prepared data."""
    prediction_request['EJ'][0]= pd.NA
    prediction_request['EJ'][1] = 'C'
    return prediction_request
#prediction_request = dirtify_prediction(prediction_request)

def replace_column_values(dataframe, name_of_columns_to_replace=[], replacement_dictionary={}):
    """Replaces values acording to the replacemente dictionary in multiple columns. of the dataframe"""
    for column_name in name_of_columns_to_replace:
        dataframe[column_name] = dataframe[column_name].replace(replacement_dictionary)
    return dataframe